# <center>DISCLAIMER !!!</center>                                    

1) Вся приведенная здесь информация даётся исключительно в просветительских целях. Пользуйтесь на свой страх и риск.    
2) Нет никакой гарантии, что описанный метод и соответствующий софт будут работать с будущими версиями защиты.  
3) Всё изложение ведётся примерно в том порядке, в котором я это исследовал. Если у Вас получится лучше, я за Вас рад, но кидаться в меня какашками право же не стоит.

In [2]:
import requests as rqs
import base64
from PIL import Image
import numpy as np
import math
from bs4 import BeautifulSoup as soup 
import threading     
import time 
print("Готово !")

Готово !


Ресурс **[proxyrotator](https://www.proxyrotator.com/free-proxy-list/)** использует самодельную защиту. В отличии от **[hidemyname](https://hidemy.name/en/)** никаких джаваскриптов (проверяющих "человечность" клиента)  нас выполнять не просят. Зато формируют веб-страничку со списком прокси в непотребном для сканирования виде. Поле ip-адреса замусоривают невидимыми символами, а номер порта выводят ввиде картинки в формате **png**. И если для **[hidemyname](https://hidemy.name/en/)** ещё можно предположить, что его владельцы таким способом защищаются от Ddos-атак, то **[proxyrotator](https://www.proxyrotator.com/free-proxy-list/)** однозначно борется с халявщиками. Насколько это делается успешно, мы сейчас увидим.   

Как и с **[hidemyname](https://hidemy.name/en/)**, начнём с попыток простейшего чтения.

In [4]:
import urllib.request as urlrq
import urllib.error as err

proxyrotator_url="https://www.proxyrotator.com/free-proxy-list/"

#Попробуйте здесь закомментировать или наоборот раскомментировать некоторые заголовки и посмотрите что получится    
rq=urlrq.Request(proxyrotator_url)
rq.add_header("User-Agent", "Mozilla/5.0 (X11; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0")
#rq.add_header("Accept", "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8")
#rq.add_header("Accept-Encoding", "gzip, deflate")
#rq.add_header("Accept-Language", "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3")
#rq.add_header("Connection", "keep-alive")
try:
    res=urlrq.urlopen(rq)
    s=str(res.read().decode("utf-8"))
    file=open("proxyrotator.html", "wt")
    file.write(s)
    file.close()   
    print(res.code)
except err.URLError as e :
    s=str(e.read().decode("utf-8"))
    file=open("proxyrotator_403.html", "wt")
    file.write(s)
    file.close()   
    print(e.code)

200


Без заголовков получили статус 403 - **Forbidden** и тот же  **Forbidden** в содержимом странички. С заголовками статус 200 и нормальная (но замусоренная !!!) страничка со списком прокси. Т.е. заголовки всё-таки проверяются. Наверно чтобы заворачивать совсем уж тупых ботов.   

Как и для **[hidemyname](https://hidemy.name/en/)**,  читать будем с помощью библиотеки **requests**, а парсить html библиотекой **BeautifulSoup**.

In [5]:
# Давайте прочтём страничку,  не забыв сохранить её в файл
firefox={        # Заголовки из mozilla Firefox
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",    
        "Accept-Encoding": "gzip, deflate", 
        "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
        "Connection": "keep-alive",
        "Host": "hidemy.name",
        "Referer": "https://hidemy.name/en/",        
        "Upgrade-Insecure-Requests": "1",    
        "User-Agent":"Mozilla/5.0 (X11; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0"   
        }
proxyrotator_url="https://www.proxyrotator.com/free-proxy-list/
rsp=rqs.get(proxyrotator_url, headers=firefox)         
html=str(rsp.content.decode(rsp.encoding))       
file=open("proxyrotator.html", "wt")               
file.write(html)
file.close()
print(rsp.status_code)
bsObj=soup(html, "lxml") 

200


Взглянув на html-код видим, что таблица на страничке только одна. В начале каждой строки таблицы задаётся куча стилей. Ещё одна такая куча стилей идёт в начале странички. Причем главным образом эти стили проделывают с отображением текста всякие непотребства. Например выставляют **display** в **none**, или **opacity** в 0, или задают абсолютную позицию, сдвигая координаты в отрицательную область... Короче всячески извращаются. Единственный вариант из всего виденного, не мешающий отображению, это **display:inline**. Понятно, что отображаться будут только те символы, стиль которых видим. Отсюда идея как можно вычистить мусор из поля ip-адреса. Из таблицы стилей извлекаем видимые стили. Для каждого символа ip-адреса смотрим, есть ли его стиль в таблице видимых. Если есть, принимаем символ. Если нет - отбрасываем. Необходимо только помнить, что локальная таблица стилей (в начале строки) приоритетнее глобальной (в начале страницы). Поэтому из локальной таблицы нужно вытаскивать ещё и невидимые стили. Если стиль видим в глобальной таблице, но невидим в локальной, он будет невидимым. Напишем функцию, вытаскивающую видимые и невидимые стили из таблиц.

In [15]:
def getVisible(bsObj):
    visible=["inline"]           # это всегда видимый стиль
    invisible=["none"]           # это всегда невидимый стиль
    styles=bsObj.find("style").contents[0].split("\n.")
    for i in range(1, len(styles)):
        s=styles[i]
        if 'inline' in s:
            s=s[:s.index('{')].strip()
            visible.append(s)
        else:
            s=s[:s.index('{')].strip()
            invisible.append(s)            
    return visible, invisible

# Проверяем
vis, invis=getVisible(bsObj)
vis

['none',
 'sSaZZIsCjTQsDKC9yNCFOrrEPa6rjaPJNdNzhm9HZblMF',
 'Yq-UVOxaJjmZZtdCtdZt7tsgReGOCktl2jORxAdHNTTNNc1GnD7f5',
 'Vgy-rpxoBitnARUzhIiNb3DtdFYr',
 'XRarRfzKqZv-vce02CPBognP9wSNgW000pGcx4eNIKTtwc7aINdOD2h',
 'xMzGfrEOcOwVrFnI7ijLKsYsb744pGt1',
 'YRdXEVkIleoQeLVG0cJiIe37glFPwErzfmPlMwrPd4MD',
 'LKEYxBjxPaZPptJzTYCDpGYK0TSMcj073tYyf0O4mn6TP6arI',
 'MhGfpBKMKpbNFnyPJzpgNkigENSwHoIHRi90bRRhSOx4hwIJpdY',
 'mxUppTLgXtVpWevH38MrI1RAY',
 'jNPLxriRL-avdOf5Aw0Dc7tMXM',
 'KBSgHXYdlONXKbuIXx6a3MCSi3AEziN4kKyzzTic0iJcSZwc0j',
 'KKhvuxqHcJMtqaQ7JL3R520sO6SwrB8bzA',
 'mNcNQeOElqlwGvxtaiRaHiLez8dheIHSyj1J4yyf3shXoTCcO05zNJR',
 'yclBqlKgtGVJyxC8mox9fGrJaLR48OwE679Rm0Do8XtK9wK2F',
 'gT-akVPXjLkDbFLey85Prg8FC5wyjSxKRidFaGojZPWXjP1d5i',
 'mHNMHkxMwLnhJfGiSKD3JYgNG',
 'OiuFavbeuJopfNNJOeirb2ycJBiGmNRZTlGtA',
 'jevhThYVSUqprNq3k7TnPY2tXrJWwKI',
 'wERiZCQCFPuAuoJ2Nwz60j13x']

Посмотрите код странички и убедитесь, что из глобальной таблицы стилей (которая в начале страницы), вытащены все видимые стили.   
Теперь можно попытаться вычистить мусор из ip-адреса. Но поскольку всё равно нужна глобальная таблица видимых стилей, имеет смысл сразу начать писать функцию парсинга странички, которую мы будем совершенствовать от версии к версии.

In [27]:
def parseIP(tr, globTab):
    ip=""
    vis, invis=getVisible(tr)                            # видимые и невидимые стили
    syms=tr.find_all("td")[1].contents                   # получаем весь текст с мусором
    for sym in syms:
        sym=str(sym)
        if sym[0]=='<':                                  # если начинается тегом
            val=sym[sym.index('>')+1:sym.index('</')]    # вытаскиваем значение
            style=sym[sym.index('"')+1:sym.rindex('"')]  # вытаскиваем стиль
            if ':' in style:
                style=style[style.index(':')+1:]
            # условие означает, что стиль либо локально видим, либо видим глобально и не отменен локально    
            if ((style in globTab) and (not style in invis)) or (style in vis):
                ip=ip+val
    return ip
    
def parseTR1(tr, globTab):
    ip=parseIP(tr, globTab)
    return ip

def parsePage1(bsObj):
    proxies=[]
    globTab, invis=getVisible(bsObj)
    table=bsObj.find("table")
    trs=table.find_all("tr")
    for i in range(1, len(trs)):
        proxies.append(parseTR1(trs[i], globTab))
    return proxies

# Проверяем
proxies=parsePage1(bsObj)
print(len(proxies))
proxies

25


['103.90.206.85',
 '103.60.137.2',
 '117.206.150.233',
 '1.197.11.139',
 '95.9.150.33',
 '190.211.115.66',
 '149.28.147.8',
 '77.232.167.200',
 '139.59.108.226',
 '103.84.36.130',
 '139.194.43.133',
 '103.253.27.53',
 '181.210.134.59',
 '103.60.137.2',
 '159.192.141.181',
 '36.67.24.109',
 '206.54.183.165',
 '203.81.93.238',
 '185.238.239.81',
 '103.81.13.17',
 '200.69.70.134',
 '41.92.186.156',
 '139.255.25.106',
 '185.238.239.16',
 '199.189.86.111']

Откройте сохраненный файл в браузере, и убедитесь, что ip-адреса там видны именно такие.

Разбираемся с картинками. Картинки лежат в той же таблице в виде кодированных Base-64 бинарных данных. Для распознавания картинки нам не надо обучать нейронную сетку, тут шрифт один и тот же. Можно гимпом (фотошопом если под Windows) нарезать картинок всех 10 цифр, а потом сдвигая потихоньку окно, сравнивать изображение в окне с каждой из цифр. Какая из 10 цифр наиболее похожа, ту и принимаем. Сравнивать картинки можно вычисляя оценку сходства как нормированное скалярное произведение картинок, представляя их длинными-длинными векторами. В математике это называется косинус угла. В цифровой обработке сигналов - свертка. Давайте сначала сохраним приходящие на страничке картинки на диск,чтобы потом нарезать из них цифр.  

In [56]:
nPic=1
def savePictures(bsObj):
    global nPic
    res=[]
    table=bsObj.find("table")
    trs=table.find_all("tr")
    for i in range(1, len(trs)):
        tds=trs[i].find_all("td")
        s=str(tds[2])
        s=s[s.index("base64,")+len("base64")+1:s.index('"/')].strip()
        png=base64.b64decode(s)
        name="./img/pic_"+str(nPic)+".png"
        file=open(name, "wb")        
        file.write(png)
        file.close()
        nPic=nPic+1
        
# Запускаем и видим в подкаталоге img 25 картинок со странички
savePictures(bsObj)

Питон вообще-то змеюка медленная, как и положено ползучему гаду. А считать скалярные произведения это труд. Только для одной позиции окна и одной цифры надо перемножить два вектора размером 189 ! Для того чтобы ускорить такие вычисления, существует замечательная библиотека работы с массивами - **numpy** - numerical python. Кроме быстрых численных вычислений, она позволяет ещё быстро вырезать окна из многомерных массивов, что для нас тоже очень полезно. Поэтому все вычисления, связанные с картинками будем делать с помощью **numpy**. 

In [152]:
# Попытаемся загрузить и преобразовать изображение в массив numpy
pic=9                                                      # номер картинки
img=Image.open("./img/"+str(pic)+".png").convert('L')      # грузим и преобразуем в серый без альфа-канала 
w, h=img.size                                              # получаем размер
a=np.array(img)                                            # в numpy
a=a.reshape((w*h, ))                                       # преобразуем в одномерный
a=a*1.0                                                    # преобразуем в плавающую точку
a=a-a.sum()/(w*h)                                          # приводим среднее значение к нулю
a=a/math.sqrt(a.dot(a))                                    # нормируем
print(str(w)+", "+str(h))                                  # показываем размер
img                                                        # показываем картинку

9, 21


In [222]:
# Теперь сделаем всё то же самое, но только 10 раз, сохраняя результаты в массиве цифр
digits=[]             # здесь у нас хранятся цифры в виде [массив_numpy, ширина]
def loadDigits():
    global digits
    for num in range(0, 10):
        img=img=Image.open("./img/"+str(num)+".png").convert('L') # грузим и преобразуем в серый без альфа
        w, h=img.size                                             # получаем размер
        a=np.array(img)                                           # в numpy
        a=a.reshape((w*h, ))                                      # преобразуем в одномерный
        a=a*1.0                                                   # преобразуем в плавающую точку
        a=a-a.sum()/(w*h)                                         # приводим среднее значение к нулю
        a=a/math.sqrt(a.dot(a))                                   # нормируем
        digits.append([a, w, h])

# Проверяем
loadDigits()
a=digits[6]
print(a[1])
print(str(a[0].sum())+" должно быть около нуля")
print(str(a[0].dot(a[0]))+" должно быть около 1")

9
-8.881784197001252e-16 должно быть около нуля
0.9999999999999996 должно быть около 1


Приступаем к функции, сканирующей номер порта

In [223]:
def parsePort(td):
    s=str(td)
    # получаем картинку
    s=s[s.index("base64,")+len("base64")+1:s.index('"/')].strip()  # получаем строку
    png=base64.b64decode(s)                                        # декодируем
    file=open("potr.png", "wb")                                    # пишем в файл
    file.write(png)
    file.close()
    # грузим картинку
    img=Image.open("potr.png").convert('L') # грузим и преобразуем в серый без альфа-канала
    W, H=img.size                           # получаем размер
    npimg=np.array(img)                     # преобразуем в numpy
    # Находим начало изображения цифр
    pos=1                                   # позиция окна сканирования
    for pos in range(1, W):    
        test=npimg[:, pos]                  # в тестовый массив вырезаем вертикальную линию
        if test.max() != test.min():        # двигаемся вперёд до тех пор, пока не встретятся 
            break                           # различные пиксели на одной вертикали
    # Начинаем собственно сканирование
    numb=""                                 # наше число
    eof=False                               # признак конца сканирования
    while (not eof):
        d=[]                                # массив оценок для каждой цифры
        e=[]                                # массив смещений для каждой цифры
        eof=True
        for i in range(0, 10):              # проходим по каждой цифре
            dd=[]                           # массив оценок для цифры при смещении окна
            ww=digits[i][1]                 # ширина цифры
            hh=digits[i][2]                 # высота цифры
            for j in range(-1, 2):          # пробуем каждую цифру при смещениях окна -1, 0, 1
                if pos+j+ww < W:            # проверяем, впишется ли цифра в остаток картинки по ширине
                    eof=False
                    test=npimg[:,pos+j:pos+j+ww] # вырезаем с позиции+смещение окно шириной ww
                    tt=test.copy()               # копируем, чтобы не затирать изображение
                    tt=tt.reshape((ww*hh, ))     # преобразуем в одномерный массив
                    tt=tt*1.0                    # преобразуем в плавающую точку
                    tt=tt-tt.sum()/(ww*hh)       # приводим среднее значение к нулю
                    tt=tt/math.sqrt(tt.dot(tt))  # нормируем
                    prod=tt.dot(digits[i][0])    # вычисляем оценку - скалярное произведение
                    dd.append(prod)              # сохраняем оценку
            if not eof:
                m=max(dd)                   # максимальная оценка при различных смещениях
                im=dd.index(m)              # смещение окна для этой оценки (начинается с -1)
                d.append(m)                 # сохраняем оценку и смещение
                e.append(im-1)
        # выбираем цифру с наивысшей оценкой
        if not eof:
            m=max(d)
            im=d.index(m)
            s=e[im]
            if m > 0.9:                # если оценка велика, дописываем соответствующую цифру
                numb=numb+str(im)
                pos+=digits[im][1]+s   # и продвигаем позицию окна на ширину цифры
            else:                      
                pos+=1                 # если оценка недостаточна, сдвигаем окно на 1
    return numb        

# Наше тестовое окружение, сканирующее страничку
def parseTR2(tr, globTab):
    tds=tr.find_all("td")
    ip=parseIP(tr, globTab)
    port=parsePort(tds[2])
    return [ip, port]

def parsePage2(bsObj):
    proxies=[]
    globTab, invis=getVisible(bsObj)
    table=bsObj.find("table")
    trs=table.find_all("tr")
    for i in range(1, len(trs)):
        proxies.append(parseTR2(trs[i], globTab))
    return proxies

# Проверяем
loadDigits()
proxies=parsePage2(bsObj)
print(len(proxies))
proxies

25


[['103.90.206.85', '8080'],
 ['103.60.137.2', '10027'],
 ['117.206.150.233', '8080'],
 ['1.197.11.139', '9999'],
 ['95.9.150.33', '54749'],
 ['190.211.115.66', '43260'],
 ['149.28.147.8', '80'],
 ['77.232.167.200', '56030'],
 ['139.59.108.226', '80'],
 ['103.84.36.130', '8080'],
 ['139.194.43.133', '8080'],
 ['103.253.27.53', '8080'],
 ['181.210.134.59', '8080'],
 ['103.60.137.2', '49411'],
 ['159.192.141.181', '8080'],
 ['36.67.24.109', '37641'],
 ['206.54.183.165', '4433'],
 ['203.81.93.238', '8080'],
 ['185.238.239.81', '8090'],
 ['103.81.13.17', '57803'],
 ['200.69.70.134', '999'],
 ['41.92.186.156', '37950'],
 ['139.255.25.106', '80'],
 ['185.238.239.16', '8090'],
 ['199.189.86.111', '9801']]

Глядим на страничку в браузере и убеждаемся, что и ip-адреса и номера портов прочитаны верно.  

На этом с защитой закончено. Однако к сожалению **[proxyrotator](https://www.proxyrotator.com/free-proxy-list/)**, в отличии от **[hidemyname](https://hidemy.name/en/)**, не имеет функции фильтрации списка. А она может быть полезной. Поэтому желательно вытаскивать не только ip-адрес и номер порта, но и другую информацию. Хотя бы о стране и скорости (протокол и анонимность на **proxyrotator** по-моему вообще не меняются.). Это позволит применить какую-то фильтрацию уже при формировании списка на выходе приложения. Давайте это сделаем: 

In [225]:
# Парсит скорость
def parseSpeed(td):
    ii=td.find_all("i")
    s=len(ii)
    for i in ii:
        if i.attrs["class"][1] == "fa-star-o" :
            s-=1
    return s
        
# Парсит страну
def parseCountry(td):
    s=td.contents[1].strip()
    if "," in s:
        s=s.split(",")[1].strip()
    return s
    
# Парсит строку таблицы
def parseTR(tr):
    result=[]
    tds=tr.find_all("td")
    result.append(parseIP(tr, globTab))              # получаем ip-адрес
    result.append(parsePort(tds[2]))                 # получаем номер порта
    result.append(parseCountry(tds[3]))              # получаем страну
    result.append(parseSpeed(tds[4]))                # получаем скорость
    result.append(tds[5].contents[0].strip())        # получаем протокол
    result.append(tds[6].contents[0].strip())        # получаем уровень анонимности
    return result
    
    
# Фильтр - пока ничего не делает, просто заглушка
def filtr(proxy):
    return [proxy[0], proxy[1]]

# Чтение странички с proxyrotator
def readPage(page):
    result=[]
    if page < 1:     # страницы там от 1 до 10
        page=1
    if page > 10:
        page=10
    rsp=rqs.get(proxyrotator_url+str(page)+'/', headers=firefox)         
    html=str(rsp.content.decode(rsp.encoding))                      
    """  При желании сохраняем файл для контроля
    file=open("proxyrotator.html", "wt")               
    file.write(html)
    file.close()
    """ 
    bsObj=soup(html, "lxml")                          # создаём объект для парсера
    globTab, invis=getVisible(bsObj)                  # получаем глобальную таблицу видимых стилей
    table=bsObj.find("tbody")                         # таблица прокси
    trs=table.find_all("tr")                          # список строк
    for tr in trs:
        r=filtr(parseTR(tr, globTab))                 # парсим все строки, фильтруя результат
        if r != None:
            result.append(r)                          # сохраняем, если результат прошел фильтр 
    return result


На этом почти всё. Осталось только собрать до кучи разбросанный по всему блокноту код. Сделаем это так же как в статье про **hidemyname**, оформив код в виде класса и вне блокнота. Отдельным файлом. При желании в блокноте можно его проверить.

In [258]:
s=[["aaa", "no"], ["bbb", "yes"]]; a=["aaa", "no"]

In [259]:
a in s

True